In [1]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from modules import simulator, geometry, coreConfig, dataHandler

In [2]:
# create a iteration of magcore dimensions
filepath = "../../data/magcore/"
filename_template = "LVDT_magcore_d{diam}_l{len}.h5"

mag_diameter = [4,5,6,7,8]
mag_length   = [4,6,8,10,12]
mag_iter = coreConfig.magnet_core_iter(mag_diameter, mag_length)

for iter in mag_iter:
    diam_input = iter[0]
    len_input = iter[1]
    diam_str  = str(iter[0]).zfill(2)
    len_str   = str(iter[1]).zfill(2)
    filename = filename_template.format(diam=diam_str, len=len_str)
    print("data will be saved to: ", filepath + filename)

    simulation_freq = 10000
    simulation_amplitude = 0.02

    Magnet_geo = geometry.def_core_geo(diam_input, len_input, 'N40')
    MiddleCoil_geo = geometry.def_coil_geo("100um", 0.1, 0, 16,  3, 18, 0)
    OuterCoil_geo  = geometry.def_coil_geo("100um", 0.1, 0, 16,  3, 18, 16)

    MC_circuit = simulator.def_circuit_prop("middlecoil", 0, 0)
    OC_upper_circuit = simulator.def_circuit_prop("outercoil_upper", simulation_freq, simulation_amplitude)
    OC_lower_circuit = simulator.def_circuit_prop("outercoil_lower", simulation_freq, -simulation_amplitude)

    simulator.def_femm_problem(signal_frequency = simulation_freq)
    simulator.build_air_geometry("Outside", 10)

    m_label = simulator.build_core_geometry(Magnet_geo, 1)
    mc_label = simulator.build_coil_geometry(MiddleCoil_geo, MC_circuit, 3,customized_material=True)
    oc_upper_label = simulator.build_coil_geometry(OuterCoil_geo, OC_upper_circuit, 4, customized_material=True )
    oc_lower_label = simulator.build_coil_geometry(OuterCoil_geo, OC_lower_circuit, 5, customized_material=True, reverse=True)

    print("Magnet label: ", m_label)
    print("Middle coil label: ", mc_label)
    print("Outer coil upper label: ", oc_upper_label)
    print("Outer coil lower label: ", oc_lower_label)
    cc_config = coreConfig.def_config(-5, 1,10)
    lvdt_voltage= simulator.def_lvdt_voltage(cc_config['steps'])
    sim_results = simulator.lvdt_simulation(moving_parts_label  = [1],
                                CC_config           = cc_config,  
                                lvdt_voltage        = lvdt_voltage, 
                                OC_upper_circuit    = OC_upper_circuit,
                                OC_lower_circuit    = OC_lower_circuit,
                                MC_circuit          =MC_circuit,
                                CC_circuit          =None)

    dataHandler.save_data(sim_results, filepath + filename)

    print("data saved to: ", filepath + filename)

data will be saved to:  ../../data/magcore/LVDT_magcore_d04_l04.h5
Magnet label:  1
Middle coil label:  3
Outer coil upper label:  4
Outer coil lower label:  5
-5
OC_upper: I= 0.020, V = 1.299414+7.719337j 
OC_lower: I= -0.020, V = -1.300353-7.720650j 
MiddleCoil: I= 0.000, V = -0.000591-0.000862j 
-4
OC_upper: I= 0.020, V = 1.299424+7.719611j 
OC_lower: I= -0.020, V = -1.300176-7.722768j 
MiddleCoil: I= 0.000, V = -0.000543-0.000789j 
-3
OC_upper: I= 0.020, V = 1.299446+7.720208j 
OC_lower: I= -0.020, V = -1.300001-7.720249j 
MiddleCoil: I= 0.000, V = -0.000454-0.000550j 
-2
OC_upper: I= 0.020, V = 1.299483+7.720345j 
OC_lower: I= -0.020, V = -1.299846-7.719717j 
MiddleCoil: I= 0.000, V = -0.000327-0.000306j 
-1
OC_upper: I= 0.020, V = 1.299538+7.720244j 
OC_lower: I= -0.020, V = -1.299717-7.719539j 
MiddleCoil: I= 0.000, V = -0.000172-0.000047j 
0
OC_upper: I= 0.020, V = 1.299615+7.720452j 
OC_lower: I= -0.020, V = -1.299614-7.719414j 
MiddleCoil: I= 0.000, V = 0.000000+0.000188j 
1


In [ ]:
class MagnetCore:
    def __init__(self, diameter:list, length:list):
        self.diameter = diameter
        self.length = length
        self.diameter_input = []
        self.length_input = []
        self.diameter_str = []
        self.length_str = []

    def __len__(self):
        return len(self.diameter_input)
    
    def magnet_core_iter(self, diameter, length):
        for diam in diameter:
            for len in length:
                yield (diam, len)
    
    def iter_para(self):
        generator = self.magnet_core_iter(self.diameter, self.length)
        for gen in generator:
            diam_input = gen[0]
            len_input = gen[1]
            diam_str  = str(gen[0]).zfill(2)
            len_str   = str(gen[1]).zfill(2)
            self.diameter_input.append(diam_input)
            self.length_input.append(len_input)
            self.diameter_str.append(diam_str)
            self.length_str.append(len_str)

        return self.diameter_input, self.length_input, self.diameter_str, self.length_str
